In [ ]:
IMPLEMENTATION OF THE CUSTOMER COUNT

DATASET : ONE VIDEO

https://mega.nz/file/wsEmmSKD#c3hl49wa37UPLKErehPNhp1CkV_ntf1R1WUeHjaXyRo


In [3]:
import cv2
from tracker import *
import pandas as pd
import numpy as np
import math
from ultralytics import YOLO


In [4]:
import os
print(os.getcwd())


C:\Users\KanugulBharath Kumar\Downloads\peoplecounteryolov8-main\peoplecounteryolov8-main


In [5]:
import os
print(os.listdir())


['.ipynb_checkpoints', 'coco.txt', 'coffee shop people count maintainance.ipynb', 'download.txt', 'main.py', 'peoplecount1', 'peoplecount1.mp4', 'tracker.py', 'Untitled.ipynb', 'Untitled1.ipynb', 'Untitled2-Copy1.ipynb', 'yolov8s.pt', '__pycache__']


In [ ]:

tracker = projection()

model = YOLO('yolov8s.pt')

area1 = [(312, 388),(289, 390),(474, 469),(497, 462)]
area2 = [(279, 392),(250,397) ,(423,477),(454, 469)]


def RGB(event, x, y, flags, param):
    
    if event == cv2.EVENT_MOUSEMOVE:
        colorsBGR = [x, y]
        print(colorsBGR)


cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap = cv2.VideoCapture('peoplecount1.mp4')

my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n")

count = 0

people_entering = {}
people_exiting = {}
entering = set()
exiting = set()

while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 2 != 0:
        continue
    frame = cv2.resize(frame, (1020, 500))
    results = model.predict(frame)
    a = results[0].boxes.data
    px = pd.DataFrame(a).astype("float")
    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'person' in c:
            list.append([x1, y1, x2, y2])
    bbox_id = tracker.update(list)
    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox

        # into the hotel
        result = cv2.pointPolygonTest(np.array(area2, np.int32), ((x4, y4)), False)
        if result >= 0:
            people_entering[id] = (x4, y4)
            cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2)
        if id in people_entering:
            result1 = cv2.pointPolygonTest(np.array(area1, np.int32), ((x4, y4)), False)
            if result1 >= 0:
                cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                cv2.circle(frame, (x4, y4), 5, (255, 0, 255), -1)
                cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, (0.5), (255, 255, 255), 1)
                entering.add(id)

    #out of hotel
        result2 = cv2.pointPolygonTest(np.array(area1, np.int32), ((x4, y4)), False)
        if result2 >= 0:
            people_exiting[id] = (x4, y4)
            cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
        if id in people_exiting:
            result3 = cv2.pointPolygonTest(np.array(area2, np.int32), ((x4, y4)), False)
            if result3 >= 0:
                cv2.rectangle(frame, (x3, y3), (x4, y4), (255, 0, 0), 2)
                cv2.circle(frame, (x4, y4), 5, (255, 0, 255), -1)
                cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, (0.5), (255, 255, 255), 1)
                exiting.add(id)

    cv2.polylines(frame, [np.array(area1, np.int32)], True, (255, 0, 0), 2)
    cv2.putText(frame, str('entryROI'), (504, 471), cv2.FONT_HERSHEY_COMPLEX, (0.5), (0, 0, 0), 1)

    cv2.polylines(frame, [np.array(area2, np.int32)], True, (255, 0, 0), 2)
    cv2.putText(frame, str('exitROI'), (466, 485), cv2.FONT_HERSHEY_COMPLEX, (0.5), (0, 0, 0), 1)
    cv2.putText(frame, "THE COFFEE SHOP", (20, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255), 2)

    print(len(entering))
    print(len(exiting))
    ccc=len(entering)
    vvv=len(exiting)
    ccc1=ccc-vvv
    cv2.putText(frame, str('Current customer count:'), (40, 120), cv2.FONT_HERSHEY_COMPLEX, (0.8), (0, 255, 255), 3)
    #cv2.putText(frame, str('Max capacity:4'), (40,147), cv2.FONT_HERSHEY_COMPLEX, (0.8), (0, 255, 255), 3)
    if ccc1>=4:
        cv2.putText(frame, str(ccc1), (80, 160), cv2.FONT_HERSHEY_COMPLEX, (0.8), (0, 0, 255), 3)
    
    else:    
        cv2.putText(frame, str(ccc1), (80, 160), cv2.FONT_HERSHEY_COMPLEX, (0.8), (255, 0, 255), 3)
    #cv2.putText(frame, str('Max capacity:4'), (40,147), cv2.FONT_HERSHEY_COMPLEX, (0.8), (0, 255, 255), 3)


    cv2.imshow("window", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



0: 320x640 1 car, 148.6ms
Speed: 11.2ms preprocess, 148.6ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
0
0

0: 320x640 1 car, 113.5ms
Speed: 0.0ms preprocess, 113.5ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
0
0

0: 320x640 1 person, 1 car, 117.8ms
Speed: 2.0ms preprocess, 117.8ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
0
0

0: 320x640 1 person, 1 car, 114.0ms
Speed: 2.0ms preprocess, 114.0ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)
0
0

0: 320x640 1 person, 1 car, 134.7ms
Speed: 2.0ms preprocess, 134.7ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
0
0

0: 320x640 1 person, 1 car, 108.6ms
Speed: 1.0ms preprocess, 108.6ms inference, 15.6ms postprocess per image at shape (1, 3, 320, 640)
0
0

0: 320x640 1 person, 2 cars, 128.5ms
Speed: 1.8ms preprocess, 128.5ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)
0
0

0: 320x640 1 person, 2 cars, 131.7ms
Speed:

In [ ]:
class projection:
    def __init__(self):
        self.center_points = {}
        self.id_count = 0


    def update(self, objects_rect):
        objects_bbs_ids = []

        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        self.center_points = new_center_points.copy()
        return objects_bbs_ids